# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'


# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Task 1: Getting  the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


- We wil name our table **song_details**.
- The primary key consisits of partition key sessionId, and clustering key itemInSession so that we can filter by this attributeslater.
- The columns of our table will be: sessionId, itemInSession, artist, songTitle, songLength.

In [10]:
song_details_table_create = "CREATE TABLE IF NOT EXISTS song_details"
song_details_table_create = song_details_table_create + "(sessionId int, itemInSession int, artist text, songTitle text, \
                                                        songLength float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(song_details_table_create)
except Exception as e:
    print(e)
    
                    

#### Insert all song details from the csv data into song_details table

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_details (sessionId, itemInSession, artist, songTitle, songLength)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Select the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [12]:
select_query_1 = "SELECT artist, songTitle, songLength \
                  FROM song_details \
                  WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(select_query_1)
except Exception as e:
    print(e)
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.songtitle, "\nSong Length: "+str(row.songlength))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


### Task 2: Getting only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


- We will name our table artist_details.
- The primary key consists of: 
    - Composite partition key (userId, sessionId). The reason of this choice is that the sessionId which belongs to the userId will be in the same node as userId.
    - Clustering key itemInSession so that our results are order by it.
- The columns of our table will be: userId, sessionId, itemInSession, firstName, lastName.

In [13]:
artist_details_table_create = "CREATE TABLE IF NOT EXISTS artist_details"
artist_details_table_create = artist_details_table_create + "(userId int, sessionId int, itemInSession int, \
                                                            artist text, songTitle text, firstName text, lastName text, \
                                                             PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(artist_details_table_create)
except Exception as e:
    print(e)
                    

#### Insert all artist details from csv data into artist_details table

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_details (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Select the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
select_query_2 = "SELECT artist, songTitle, firstName, lastName FROM artist_details \
                    WHERE userId = 10 \
                    AND sessionId = 182 "
try:
    rows = session.execute(select_query_2)
except Exception as e:
    print(e)
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.songtitle, "\nUser First Name: "+row.firstname, "\nUser Last Name: "+row.lastname)

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
User First Name: Sylvie 
User Last Name: Cruz


### Task 3:  Getting every user name (first and last) in  music app history who listened to the song 'All Hands Against His Own'


- We will name our table user_details.
- The primary key consists of partition key song, and clustering key userId. This uniquely identifies our rows.
- The columns of our table will be: songTitle, userId, firstName, lastName.

In [18]:
user_details_table_create = "CREATE TABLE IF NOT EXISTS user_details"
user_details_table_create = user_details_table_create + "(songTitle text, userId int, firstName text, lastName text, \
                                        PRIMARY KEY (songTitle, userId))"
                
try:
    session.execute(user_details_table_create)
except Exception as e:
    print(e)
                    

#### Insert all user details from csv data into user_details table

In [19]:
# Insertion for Table 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(songTitle, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Select every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'

In [20]:
select_query_3 = "SELECT firstName, lastName \
                  FROM user_details \
                  WHERE songTitle = 'All Hands Against His Own'"
try:
    rows = session.execute(select_query_3)
except Exception as e:
    print(e)
for row in rows:
    print("User First Name: "+row.firstname, "\nUser Last Name: "+row.lastname)

User First Name: Jacqueline 
User Last Name: Lynch
User First Name: Tegan 
User Last Name: Levine
User First Name: Sara 
User Last Name: Johnson


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS artist_details")
session.execute("DROP TABLE IF EXISTS user_details")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()